# Część trzecia
<div class="alert alert-block alert-danger">
<b>Uwaga:</b> Należy upewnić się że w prawym górnym rogu tego okna wybrany jest kernel "edugen" a nie "Python 3"
</div>

***
## Format VCF
***

Przyjętym standardowym formatem plików przechowującym informację o wariantach jest format <a href="http://samtools.github.io/hts-specs/VCFv4.3.pdf">VCF (Variant Call Format)</a>. W takich plikach przechowywana jest informacja o wszystkich wariantach odnalezionych przez program do variant callingu. Pliki VCF możemy podzielić na trzy sekcje:
- metadanych (@meta)
- informacje o wariatach (@fix)
- informacje o genotypie (@gt)

<img src="png/vcf.png" width="400">


In [ ]:
! head -n 50 sandbox/motherHC.vcf

***
### Region metadanych (@meta)
Znajduje się na początku pliku i zawiera informacje (poprzedzone znakami "##") dotyczące programu który wygenerował plik VCF oraz parametrów używanych przez ten program, jak również informacje na temat skrótów stoowanych w innych miejscach pliku. Dane w tej sekcji są zapisywane w kolejnych wierszach. Pozostałe regiony mają formę tabeli.

### Region informacji o wariancie (@fix)
Region ten obejmuje pierwszych 8 kolumn, które zawierają informację o konkretnym wariancie. Dane te są wspólne dla wszystkich wariantów i obejmują:
- pozycję chromosomową (kolumny "#CHROM" i "POS")
- identyfikator wariantu, np. z bazy dbSNP (kolumna "ID")
- allel referencyjny i zmieniony (kolejno kolumny "REF" i "ALT") - uwaga, liczba zmienionych alleli może być większa niż 1, sa wtedy zapisywane po przecinku
- informację o jakości danego wariantu (kolumna "QUAL")
- informację czy wariant przeszedł filtr jakości (kolumna "FILTER")

### Region informacji o genotypie (@gt)
W tym regionie znajdziemy informację związane z danym wariantem dla poszczególnych próbek. W 10 kolumnie poszczególne wartości sa oddzielone od siebie dwukropkiem, a informację co która wartość oznacza znajdują się w kolumnie 9 ("FORMAT"). Najczęsciej są to informacje o genotypie, ilości odczytów w danym miejscu czy ich jakości, a wyjaśnienie stosowanych skrótów i co oznaczają znajdziemy w regionie metadanych (oraz na stronie specyfikacji formatu VCF). Jeśli w pliku VCF opisanych jest kilka próbek, to informacje dla kolejnych próbek są zapisywane w kolejnych kolumnach. Pola które zwyczajowo znajdują się w tym regionie to:
- GT (Genotype) - genotyp w danym miejscu, oddzielony znakami "/" lub "|". Wartość 0 oznacza allel referencyjny, 1 oznacza pierwszy zmieniony allel z pola "ALT", 2 drugi allel itd. Kombinacja 0/1 oznacza heterozygotyczna zmianę, 1/1 zmianę homozygotyczną
- AD (Allel Read Depth) - liczba odczytów zawierających dany wariant
- DP (Read Depth) - ogólna liczba odczytów w danym miejscu

| #CHROM | POS    | ID    | REF | ALT | QUAL | FILTER | INFO | FORMAT   | SAMPLE_1_ID           | SAMPLE_2_ID               |
|--------|--------|-------|-----|-----|------|--------|------|----------|-----------------------|---------------------------|
| 22     | 160036 | snp_1 | A   | C   | .    | PASS   | .    | GT:AD:DP | 0/1:7:30              | 1/1:31:31                 |
| 22     | 160408 | snp_2 | T   | C   | .    | PASS   | .    | GT:AD:DP | ./.:.:.               | 0/1:10:21                 |
| 22     | 160612 | snp_2 | C   | G   | .    | PASS   | .    | GT:AD:DP | 0/1:22:53             | 1/1:41:41                 |

***
## Normalizacja plików VCF
***
Pomimo tego, że VCF jest standardowym formatem przechowywania danych o wariantach to warianty mogą być przechowywane w różnej postaci. W celu unifikacji tych informacji należy skorzystac z <a href="https://genome.sph.umich.edu/wiki/Vt">narzędzia Vt</a>. Pozwala ono na rozbicie multi-allelicznych wariantów na poszczególne wiersze. Dodatkowo ujednolica sposób zapisu wariantów typu INDEL, tak aby były przesunięte możliwie najbardziej na "lewo" oraz możliwie najbardziej "oszczędne" w zapisie.
<img src="png/Normalizacja.png">

***
Sprawdźmy najpierw czy nasz plik posiada warianty multialleliczne. Możemy użyć do tego funkcji `view` programu `Vt` i przefiltrować plik w poszukiwaniu tych wariantów.
<div class="alert alert-block alert-warning">
    <b>Zadanie:</b> Na podstawie dokumentacji programu Vt wyświetl warianty multialleliczne z próbki <b>sandbox/motherHC.vcf</b> 
</div>

***
W naszym pliku znajduje się 8 wariantów biallelicznych. Najpierw nalezy przeprowadzić dekompozycję, czyli rozbicie tych wariantów a następnie normalizację, czyli ujednolicenie zapisu wszystkich wariantów. Służą do tego kolejno funkcje `decompose` oraz `normalize`.

In [ ]:
! vt decompose -s sandbox/motherHC.vcf | vt normalize -r ref/ref.fasta - > sandbox/motherHC.vt.vcf

***
## Eksploracja plików VCF
***
Narzędziem służącym do manipulowania plikami VCF jest `bcftools`. Program ten posiada <a href="https://samtools.github.io/bcftools/bcftools.html">wiele opcji</a>, możemy na przykład filtrować warianty, wydzielać regiony czy podsumować dane. Na początek porównajmy pliki przed i po obróbce programem Vt.

<div class="alert alert-block alert-warning">
    <b>Zadanie:</b> Policz ile wariantów znajduje się w plikach <b>motherHC.vcf</b> i <b>motherHC.vt.vcf</b>. Czy w pliku źródłowym były warianty multialleliczne? Użyj poleceń bash - <i>cat</i>, <i>grep</i> i <i>wc</i> 
</div>

Możemy również podejrzeć ile jest wariantów typu snp lub indel. Służy do tego komenda `view` z argumentem `-v`.

In [ ]:
! bcftools view -v snps sandbox/motherHC.vcf | grep -v "^#" | wc -l
! bcftools view -v indels sandbox/motherHC.vcf | grep -v "^#" | wc -l

***
### Poszukiwanie wariantów wspólnych lub unikalnych
Przydatną funkcją może być poszukiwanie wariantów które występują w dwóch lub większej liczbie próbek, lub wariantów unikalnych dla danej próbki. Służy do tego komenda `isec` z pakietu `bcftools` o której więcej możemy <a href="https://samtools.github.io/bcftools/bcftools.html#isec">poczytać tutaj</a>. Domyślnie program daje wyniki w postaci wariantów unikalnych dla konkretnych próbek oraz warianty wspólne dla wszystkich. Argument `-p` pozwala określić ścieżkę zapisu plików wynikowych, a następnie podajemy ścieżkę do plików które chcemy porównać. Porównajmy pliki przed i po normalizacji programem `Vt`.

In [ ]:
! bcftools isec -p sandbox/isec sandbox/motherHC.vcf.gz sandbox/motherHC.vt.vcf.gz

Wyniki znajdziemy w folderze `sandbox/isec`, w którym utworzyły się 4 pliki, ich opis znajduje się w pliku `README.txt`

In [ ]:
! cat sandbox/isec/README.txt

Możemy bliżej przyjrzeć się plikom wynikowym i zobaczyć warianty unikalne oraz wspólne

In [ ]:
! cat sandbox/isec/0000.vcf | grep -v "##" | head -n 100

In [ ]:
! cat sandbox/isec/0000.vcf | grep -v "##" | wc -l

***
### Łączenie kilku plików VCF w jeden
Program `bcftools` pozwala również na łączenie kilku plików VCF w jeden. Służy do tego funkcja `merge`, która posiada <a href="https://samtools.github.io/bcftools/bcftools.html#merge">kilka opcji</a>. Połączymy pliki z przed i po normalizacji. Aby tego dokonać najpierw musimy je skompresować i utworzyć dla nich indeksy.

In [ ]:
! bgzip -c sandbox/motherHC.vcf > sandbox/motherHC.vcf.gz
! bgzip -c sandbox/motherHC.vt.vcf > sandbox/motherHC.vt.vcf.gz
! bcftools index -t sandbox/motherHC.vcf.gz
! bcftools index -t sandbox/motherHC.vt.vcf.gz

Nastepnie za pomoca argumentu `-o` podajemy miejsce i nazwę pliku wynikowego. W naszym przypadku musimy również użyć argumentu `--force-samples`, ponieważ nasze pliki mają samą nazwę zapisaną w środku pliku VCF.
<div class="alert alert-block alert-warning">
    <b>Zadanie:</b> Połącz pliki <b>motherHC.vcf</b> i <b>motherHC.vt.vcf</b> w jeden plik VCF. Następnie zbadaj jak wygląda zapis wariantów które zostały znormalizowane.  
</div>

***
# Adnotacja plików VCF
Nie wszystkie warianty są dla nas interesujące. Znakomita większość z nich to warianty w regionach niekodujących, bądź warianty bez wpływu na kodowane białko. Informacja w plikach VCF które otrzymujemy najczęsciej pozbawiona jest podstawowych danych, np. o nazwie genu w obrębie którego lokalizuje sie dany wariant, rodzaju zmiany, częstości wariantu w populacji itd. Dlatego pliki VCF należy jeszcze zaadnotować. Od nas zależy jakiego rodzaju dane i z jakich baz danych chcemy je pozyskać. Istnieje wiele programów do adnotacji, do najpopularniejszych należą <a href="https://doc-openbio.readthedocs.io/projects/annovar/en/latest/">Annovar</a> oraz <a href="http://grch37.ensembl.org/info/docs/tools/vep/index.html">Variant Effect Predictor</a>. Są to 

Na zajęciach możemy skorzystać z przeglądarkowej <a href="http://grch37.ensembl.org/Tools/VEP">wersji VEP</a>

***
# Programy do zarządzania przepływem danych

Najczęsciej w przypadku analizy danych genomowych mamy wiele próbek które mogą wymagac odpowiedniej obróbki bioinformatycznej. Dlatego możemy stworzyć stałe, okreslone zadania i skrypty. Przydaja się do tego programy pomagające w przepływie danych i pracy, które pozwalają na automatyzację procesów, a także tworzenie powtarzalnych i skalowalnych analiz danych. Pozwalają na konfigurację, kontrolę jakości i monitorowanie sekwencji zadań. Z ich pomocą możemy tworzyć gotowe schematy reguł co ułatwi nam uruchomienie wybranych procesów w dowolnym momencie.

Przykładem takiego narzędzie może być <a href="https://snakemake.readthedocs.io/en/stable/">Snakemake</a>. Pozwala on na stworzenie reguł, które opisują co należy zrobić, aby z danych plików wejściowych, przeprowadzając konkretne działania, powstały określone pliki wyjściowe.

**Graficzny przykład reguły:**<br>
<img src="png/snakemake.png">


**Przykład reguły:**

rule bwa:<br>
    input:<br>
        "data/genome.fa",<br>
        "data/samples/A.fastq"<br>
    output:<br>
        "mapped/A.bam"<br>
    conda:<br>
        "envs/mapping.yaml"<br>
    shell:<br>
        "bwa mem {input} | samtools view -Sb - > {output}"<br>

***
# Analiza danych pacjentów z rzadkimi chorobami monogenowymi

Kolejne kroki w priorytetyzacji wariantów:
1. Odfiltrowanie wariantów nie przechodzących filtru jakości (kolumna F - "FILTER")
1. Odfiltrowanie wariantów nie kodujących (intron, intergenic, synonymous, upstream, downstream, UTR) - (kolumna O - "Consequence")
1. Odfiltrowanie wariantów częstych (powyżej 0.1% w populacji) - (kolumna AB - "gnomAD_AF", kolumna BR - "exome_af")
1. Analiza wariantów pod względem dziedziczenia (kolumna R - "inheritanceOMIM"), zygotyczności wariantu (kolumna M - "Zyg") oraz fenotypu (kolumna S - "phenotypeOMIM")

***
### Opis kolumn w plikach do analizy

| Nazwa   kolumny        | Opis                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|------------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Sex                    | Płeć                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| Chr                    | Chromosom na którym   zlokalizowany jest wariant                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
| POS                    | Pozycja chromosomowa danego   wariantu                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
| Ref                    | Nukleotyd(y) który znajduje   się w danej pozycji chromosomowej w sekwencji referencyjnej                                                                                                                                                                                                                                                                                                                                                                                                                                                |
| Alt                    | Zmieniony nukleotyd(y)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
| FILTER                 | Czy dany wariant przechodzi   filtr jakości GATK                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
| Qual                   | Wartość   okreslająca prawdopodobieństwo że w danym miejscu doszło do mutacji                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
| GnomAD_url             | Link do wariantu w bazie   gnomAD v3                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| VarSome_url            | Link do wariantu w bazie   VarSome                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| AD                     | Allelic Depth - Liczba   odczytów dla zmienionego nukleotydu                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| DP                     | Read Depth - Łączna liczba   odczytów w danym miejscu                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| VAF                    | Variant Allele Frequency -   Stosunek liczby zmienionych odczytów względem referencji (AD/DP)                                                                                                                                                                                                                                                                                                                                                                                                                                            |
| Zyg                    | Zygotyczność danego wariantu                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| Symbol                 | Nazwa genu                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
| Consequence            | Konsekwencja   dla danego wariantu (np.. missense, synonymous)                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
| Impact                 | Potencjalny wpływ wariantu na   kodowane białko. Każda konsekwencja posiada przypisany potencjalny wpływ                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| rs_dbSNP               | Identyfikator wariantu w bazie   dbSNP                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
| inheritanceOMIM        | Dziedziczenie dla danego genu   wg bazy OMIM                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| phenotypeOMIM          | Fenotyp(y) przypisane do   danego genu wg bazy OMIM                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
| OMIM_url               | Link do bazy OMIM dla danego   genu                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
| HGVS                   | Zapis wariantu zgodny z HGVS   (jeżeli w VEP jest przypisana dla danego wariantu sekwencja RefSeq (NM_) to   jest ona użyta, jeżeli nie, uzywany jest transkrypt wg Ensembl)                                                                                                                                                                                                                                                                                                                                                             |
| key                    | Opis wariantu w formie   chromosom:pozycja chromosomowa_allel referencyjny>allel zmieniony                                                                                                                                                                                                                                                                                                                                                                                                                                               |
| Position               | Pozycja danego wariantu w   genie - tzn. dla wariantów eksomowych - numer eksonu i pozycja nukleotydu w   sekwencji kodującej, dla wariantów intronowych numer intronu                                                                                                                                                                                                                                                                                                                                                                   |
| CLNSIG                 | Ocena patogenności wariantu wg   ClinVar                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| CLNDN                  | Przypisana dla wariantu   jednostka chorobowa/fenotyp                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| CLNREVSTAT             | Status oceny   danego wariantu wg ClinVar                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
| ClinVar_url            | Link do wariantu w bazie   ClinVar                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| gnomAD_AF              | Częstotliwość wariantu w bazie   gnomAD v3 dla wszystkich próbek z tej bazy (71702 genomy)                                                                                                                                                                                                                                                                                                                                                                                                                                               |
| gnomAD_AF_NFE          | Częstotliwość wariantu w bazie   gnomAD v3 dla pacjentów z populacji europejskiej (z wyłączeniem Finów)                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| gnomAD_AF_FIN          | Częstotliwość wariantu w bazie   gnomAD v3 dla Finów                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| gnomAD_AF_AFR          | Częstotliwość wariantu w bazie   gnomAD v3 dla pacjentów z populacji afrykańskiej                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
| gnomAD_AF_AMR          | Częstotliwość wariantu w bazie   gnomAD v3 dla pacjentów z populacji amerykańskiej                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| gnomAD_AF_ASJ          | Częstotliwość wariantu w bazie   gnomAD v3 dla pacjentów z populacji żydów aszkenazyjskich                                                                                                                                                                                                                                                                                                                                                                                                                                               |
| gnomAD_AF_EAS          | Częstotliwość wariantu w bazie   gnomAD v3 dla pacjentów z populacji wschodnioazjatyckiej                                                                                                                                                                                                                                                                                                                                                                                                                                                |
| gnomAD_AF_SAS          | Częstotliwość wariantu w bazie   gnomAD v3 dla pacjentów z populacji południowoazjatyckiej                                                                                                                                                                                                                                                                                                                                                                                                                                               |
| gnomAD_AF_AMI          | Częstotliwość wariantu w bazie   gnomAD v3 dla pacjentów z populacji Amiszów                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| gnomAD_AF_OTH          | Częstotliwość wariantu w bazie   gnomAD v3 dla pacjentów z pozostałych populacji                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
| gnomAD_nhomalt         | Liczba wariantów w układzie   homozygotycznym w bazie gnomAD v3 dla danego wariantu                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
| SpliceAI_pred          | Predykcja wpływu wariantu na   miejsce splicingowe, obliczona za pomocą sieci neuronowych przez Illuminę.   Format zapisu danych to SYMBOL\|DS_AG\|DS_AL\|DS_DG\|DS_DL\|DP_AG\|DP_AL\|DP_DG\|DP_DL. Delta Score (DS) jest obliczony dla miejsc akceptorowych i   rozpatruje czy dane miejsce może zostać aktywowane (Acceptor Gain - AG) lub   utracone (Acceptor Loss - AL), podobnie w przypadku miejsc donowoych (Donor   Gain - DG, Donor Loss - DL). Podawana jest też pozycja Delta Position (DP)   dla poszczególnych przypadków. |
| SpliceAI_cutoff        | Przyjmuje wartość PASS jeżeli   którykolwiek Delta Score przekracza próg 0.5, FAIL jeżeli żaden nie jest powyżej 0.5                                                                                                                                                                                                                                                                                                                                                                                                                     |
| LoF                    | Ocena   nietolerancji na warianty LoF (Loss of function), im mniejszy wynik tym   bardziej podatny jest dany gen na tego typu zmiany                                                                                                                                                                                                                                                                                                                                                                                                     |
| pLI                    | Analogicznie do wcześniejszej   kolumny, ale tutaj im wyższy wynik tym bardziej podatny gen na zmiany typu   LoF.                                                                                                                                                                                                                                                                                                                                                                                                                        |
| SIFT                   | D: Deleterious, T: tolerated                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| SIFT4G_pred            | D: Deleterious, T: tolerated. Kilka wartości odnosi się do poszczególnych transkryptów   danego genu                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| PolyPhen               | Predykcja PolyPhen, opis   słowny (benign itd.)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
| Polyphen2_HDIV_pred    | D: Probably damaging, P:   possibly damaging, B: benign                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| MutationTaster_pred    |  A:   disease_causing_automatic, D: disease_causing, N: polymorphism (probably   harmless), P: polymorphism_automatic (known to be harmless)                                                                                                                                                                                                                                                                                                                                                                                             |
| MutationAssessor_pred  | H: high, M: medium, L: low, N:   neutral, H/M means functional and L/N means non-functional                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| BayesDel_addAF_pred    | D: Damaging, T: Tolerated                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
| BayesDel_noAF_pred     | D: Damaging, T: Tolerated                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
| LIST-S2_pred           | D: Damaging, T: Tolerated                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
| fathmm-MKL_coding_pred | D: Deleterious, T: Tolerated                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| fathmm-XF_coding_pred  | D: Deleterious, T: Tolerated                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| DEOGEN2_pred           | D: Damaging, T: Tolerated                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
| LRT_pred               | D: Deleterious, N: Neutral, U:   Unknown                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| M-CAP_pred             | D: Damaging, T: Tolerated                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
| MetaLR_pred            | D: Deleterious, T: Tolerated                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| MetaSVM_pred           | D: Deleterious, T: Tolerated                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| PROVEAN_pred           | D: Deleterious, N: Neutral                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
| PrimateAI_pred         | D: Damaging, T: Tolerated                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
| CADD_phred             | Wartość CADD wyliczana na   podstawie wielu różnych predykcji. Im wyższa, tym większe prawdopodobieństwo   że wariant jest uszkadzający. Nie ma ściśle ustalonej   wartości progowej, jednak niektóre źródła (np. Ensembl) sugeruje granicę 30   (powyżej uszkadzające, poniżej łagodne)                                                                                                                                                                                                                                                 |
| Feature                | Numer transkryptu w bazie   Ensembl                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
| ENSP                   | Numer sekwencji białkowej wg   bazy Ensembl                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| v_hom_exome            | Liczba wariantów w układzie   homozygotycznym dla eksomów w bazie IMID dla danego wariantu                                                                                                                                                                                                                                                                                                                                                                                                                                               |
| v_hom_pass_exome       | Liczba wariantów w układzie   homozygotycznym dla eksomów, przechodzących filtr jakości (PASS) w bazie IMID   dla danego wariantu                                                                                                                                                                                                                                                                                                                                                                                                        |
| v_het_exome            | Liczba wariantów w układzie   heterozygotycznym dla eksomów w bazie IMID dla danego wariantu                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| v_het_pass_exome       | Liczba wariantów w układzie   heterozygotycznym dla eksomów, przechodzących filtr jakości (PASS) w bazie   IMID dla danego wariantu                                                                                                                                                                                                                                                                                                                                                                                                      |
| v_hem_exome            | Liczba wariantów w układzie   hemizygotycznym dla eksomów w bazie IMID dla danego wariantu                                                                                                                                                                                                                                                                                                                                                                                                                                               |
| v_hem_pass_exome       | Liczba wariantów w układzie   hemizygotycznym dla eksomów, przechodzących filtr jakości (PASS) w bazie IMID   dla danego wariantu                                                                                                                                                                                                                                                                                                                                                                                                        |
| exome_af               | Częstość wystepowania danego   wariantu we wszystkich próbkach z eksomów (uwzględniana jest zygotyczność   wariantów)                                                                                                                                                                                                                                                                                                                                                                                                                    |
| exome_af_pass          | Częstość wystepowania danego   wariantu we wszystkich próbkach z eksomów, jeżeli przechodzi filtr jakości   (uwzględniana jest zygotyczność wariantów)                                                                                                                                                                                                                                                                                                                                                                                   |
***

Pliki z adnotacjami pacjentów znajdują się w folderze `case`

### Pacjent 1
Chłopiec. Cechy kliniczne - małogłowie, opóźnienie rozwoju psychoruchowego, rozszczep podniebienia, padaczka, niedosłuch

### Pacjent 2
Dziewczynka. Cechy kliniczne - małogłowie, hipoplazja móżdżku, zaburzenia zakrętowości mózgu, dysmorfie twarzy

### Pacjent 3
Chłopiec. Podejrzenie nnerwiakowłókniakowatości typu I. Cechy kliniczne - plamy o zabarwieniu kawy z mlekiem, zmiany w obrębie tęczówki

### Pacjent 4
Chłopiec. Podejrzenie zespołu Marfana. Cechy kliniczne - wysoki wzrost, skolioza, płaskostopie